In [1]:
# Import required libraries
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
# Read in the insurance dataset
insurance = pd.read_csv("https://raw.githubusercontent.com/stedy/Machine-Learning-with-R-datasets/master/insurance.csv")

In [27]:
# Check out the insurance dataset
insurance

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


In [23]:
#check if there is 0 value
insurance.isna().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

In [5]:
insurance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [6]:
# Turn all categories into numbers
insurance_one_hot = pd.get_dummies(insurance)
insurance_one_hot.head() # view the converted columns

,age,bmi,children,charges,sex_female,sex_male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest
0,19,27.900,0,16884.92400,1,0,0,1,0,0,0,1
1,18,33.770,1,1725.55230,0,1,1,0,0,0,1,0
2,28,33.000,3,4449.46200,0,1,1,0,0,0,1,0
3,33,22.705,0,21984.47061,0,1,1,0,0,1,0,0
4,32,28.880,0,3866.85520,0,1,1,0,0,1,0,0


In [7]:
# Create X & y values
X = insurance_one_hot.drop("charges", axis=1)
y = insurance_one_hot["charges"]

In [8]:
# Create training and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.2, 
                                                    random_state=42) # set random state for reproducible splits

In [9]:
X_train

,age,bmi,children,sex_female,sex_male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest
560,46,19.950,2,1,0,1,0,0,1,0,0
1285,47,24.320,0,1,0,1,0,1,0,0,0
1142,52,24.860,0,1,0,1,0,0,0,1,0
969,39,34.320,5,1,0,1,0,0,0,1,0
486,54,21.470,3,1,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
1095,18,31.350,4,1,0,1,0,1,0,0,0
1130,39,23.870,5,1,0,1,0,0,0,1,0
1294,58,25.175,0,0,1,1,0,1,0,0,0
860,37,47.600,2,1,0,0,1,0,0,0,1


Try using RidgeCV

In [24]:
from sklearn.linear_model import RidgeCV
import numpy as np

rdg = RidgeCV(alphas=[1e-3, 1e-2, 1e-1, 1])
rdg.fit(X_train, y_train)
RidgeCV()

RidgeCV(alphas=array([ 0.1,  1. , 10. ]))

In [25]:
rdg_preds = rdg.predict(X_test)

In [26]:
from sklearn.metrics import mean_squared_error

lin_mse = mean_squared_error(rdg_preds, y_test)
rdg_rmse = np.sqrt(lin_mse)
rdg_rmse

5798.271036065023

Try Using Neural Networks

In [21]:
# Set random seed
tf.random.set_seed(42)

# Create a new model 
insurance_model = tf.keras.Sequential([
  tf.keras.layers.Dense(units=11, activation='relu',), 
  tf.keras.layers.Dense(units=50, activation='relu',),
  tf.keras.layers.Dense(units=50, activation='relu',),
  tf.keras.layers.Dense(units=10, activation='relu',),
  tf.keras.layers.Dense(units=9, activation='relu',),
  tf.keras.layers.Dense(units=8, activation='relu',),
  tf.keras.layers.Dense(units=1, activation='relu',),

])

# Compile the model
insurance_model.compile(loss=tf.keras.losses.mae,
                        optimizer=tf.keras.optimizers.Adam(),
                        metrics=['mae'])

# Fit the model
insurance_model.fit(X_train, y_train, epochs=300)

Epoch 1/300
34/34 [==============================] - 1s 2ms/step - loss: 13343.3115 - mae: 13343.3115
Epoch 2/300
34/34 [==============================] - 0s 1ms/step - loss: 13301.0615 - mae: 13301.0615
Epoch 3/300
34/34 [==============================] - 0s 1ms/step - loss: 12834.6973 - mae: 12834.6973
Epoch 4/300
34/34 [==============================] - 0s 1ms/step - loss: 9817.3359 - mae: 9817.3359
Epoch 5/300
34/34 [==============================] - 0s 1ms/step - loss: 7681.9502 - mae: 7681.9502
Epoch 6/300
34/34 [==============================] - 0s 1ms/step - loss: 7532.1284 - mae: 7532.1284
Epoch 7/300
34/34 [==============================] - 0s 1ms/step - loss: 7433.5986 - mae: 7433.5986
Epoch 8/300
34/34 [==============================] - 0s 1ms/step - loss: 7345.0537 - mae: 7345.0537
Epoch 9/300
34/34 [==============================] - 0s 1ms/step - loss: 7267.9038 - mae: 7267.9038
Epoch 10/300
34/34 [==============================] - 0s 1ms/step - loss: 7190.6538 - mae: 719

In [28]:
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

In [29]:
models = {
    'SVR':SVR(),
    'XGBRegressor':XGBRegressor(),
    'Ridge':Ridge(),
    'ElasticNet':ElasticNet(),
    'SGDRegressor':SGDRegressor(),
    'BayesianRidge':BayesianRidge(),
    'LinearRegression':LinearRegression(),
    'RandomForestRegressor':RandomForestRegressor()
}

In [30]:
from sklearn.metrics import mean_squared_error


In [32]:
X_train

,age,bmi,children,sex_female,sex_male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest
560,46,19.950,2,1,0,1,0,0,1,0,0
1285,47,24.320,0,1,0,1,0,1,0,0,0
1142,52,24.860,0,1,0,1,0,0,0,1,0
969,39,34.320,5,1,0,1,0,0,0,1,0
486,54,21.470,3,1,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
1095,18,31.350,4,1,0,1,0,1,0,0,0
1130,39,23.870,5,1,0,1,0,0,0,1,0
1294,58,25.175,0,0,1,1,0,1,0,0,0
860,37,47.600,2,1,0,0,1,0,0,0,1


In [33]:
y_train

560      9193.83850
1285     8534.67180
1142    27117.99378
969      8596.82780
486     12475.35130
           ...     
1095     4561.18850
1130     8582.30230
1294    11931.12525
860     46113.51100
1126    10214.63600
Name: charges, Length: 1070, dtype: float64

In [35]:
model_results = []
model_names = []

# training the model with function
for name,model in models.items():
    a = model.fit(X_train, y_train)
    predicted = a.predict(X_test)
    score = np.sqrt(mean_squared_error(y_test, predicted))
    model_results.append(score)
    model_names.append(name)
    
    #creating dataframe
    df_results = pd.DataFrame([model_names,model_results])
    df_results = df_results.transpose()
    df_results = df_results.rename(columns={0:'Model',1:'RMSE'}).sort_values(by='RMSE',ascending=False)
    
print(df_results)

[05:17:18] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
                   Model                RMSE
4           SGDRegressor  11808734818.987705
0                    SVR        12903.571294
3             ElasticNet         8426.473145
5          BayesianRidge         5798.425998
2                  Ridge         5798.271036
6       LinearRegression         5796.284659
7  RandomForestRegressor         4680.403905
1           XGBRegressor         4330.751345
